In [1]:
# Importing Library


import pandas as pd
import numpy as np
from sklearn.cross_validation import StratifiedKFold
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
# import pandas as pd
# import numpy as np
import xgboost as xgb
from xgboost.sklearn import XGBClassifier
from sklearn import cross_validation, metrics   #Additional scklearn functions
from sklearn.grid_search import GridSearchCV   #Perforing grid search

import matplotlib.pylab as plt
%matplotlib inline
from matplotlib.pylab import rcParams
rcParams['figure.figsize'] = 12, 4

pd.set_option('display.max_columns', 1000)

train = pd.read_csv('train_data.csv')
test = pd.read_csv('test_data.csv')

print('The train data has {} rows and {} columns'.format(train.shape[0], train.shape[1]))
print('The test data has {} rows and {} columns'.format(test.shape[0], test.shape[1]))

C:\Users\admin\Anaconda3\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
C:\Users\admin\Anaconda3\lib\site-packages\sklearn\grid_search.py:42: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


The train data has 169307 rows and 43 columns
The test data has 91166 rows and 42 columns


In [2]:
# Data pre-processing

# Here I have removed all the rows with targe = 2 as i statastically I didnt didnot find any significant of it.
# Because eitherway over 99% data as target 2 are being missclasified in my every previous models.  

trainnew = train[train['target'] != 2]
train = trainnew
feature_names = [x for x in train.columns if x not in ['connection_id','target']]
target = train['target']
print(train.shape)

(132453, 43)


In [4]:
# modelfit function is created to find the best possible no. of estimators (n_estimators) using xgb.cv

def modelfit(alg, dytrain, predictors,target,useTrainCV=True, cv_folds=5, early_stopping_rounds=50):
    
    if useTrainCV:
        xgb_param = alg.get_xgb_params()
        xgtrain = xgb.DMatrix(dytrain[predictors].values, label=target.values)
        cvresult = xgb.cv(xgb_param, xgtrain, num_boost_round=alg.get_params()['n_estimators'], nfold=5,
            metrics='error', early_stopping_rounds=early_stopping_rounds)
        alg.set_params(n_estimators=cvresult.shape[0])

In [5]:
# xgboost classifier with standard paramerters and wthen using modelfit function to tune n_estimator
# here the modelfit output will be a classifier with tuned n_estimators
from xgboost.sklearn import XGBClassifier
xgb1 = XGBClassifier(
 learning_rate =0.1,
 n_estimators=1000,
 max_depth=5,
 silent = False,
 min_child_weight=1,
 gamma=0,
 subsample=0.8,
 colsample_bytree=0.3,
 objective= 'binary:logistic',
 nthread=4,
 scale_pos_weight=1,
 seed=27)
#modelfit(xgb1,train_feats1,target)
modelfit(xgb1,train,feature_names,target)

In [6]:
# all the parameters of xgboost classifier

xgb1.get_params()

{'base_score': 0.5,
 'colsample_bylevel': 1,
 'colsample_bytree': 0.3,
 'gamma': 0,
 'learning_rate': 0.1,
 'max_delta_step': 0,
 'max_depth': 5,
 'min_child_weight': 1,
 'missing': None,
 'n_estimators': 193,
 'nthread': 4,
 'objective': 'binary:logistic',
 'reg_alpha': 0,
 'reg_lambda': 1,
 'scale_pos_weight': 1,
 'seed': 27,
 'silent': False,
 'subsample': 0.8}

In [7]:
# pickling the classifier. I have already done that so commenting out the code.
# import pickle

# pickle.dump(xgb1,open("01xgb1.pickle","wb"))
# loaded_model = pickle.load(open('01xgb1.pickle','rb'))

In [10]:
#  fitting the model on modified training data

xgb1.fit(train[feature_names],target)

XGBClassifier(base_score=0.5, colsample_bylevel=1, colsample_bytree=0.3,
       gamma=0, learning_rate=0.1, max_delta_step=0, max_depth=5,
       min_child_weight=1, missing=None, n_estimators=193, nthread=4,
       objective='binary:logistic', reg_alpha=0, reg_lambda=1,
       scale_pos_weight=1, seed=27, silent=False, subsample=0.8)

In [11]:
# predicting and saving the output csv file

pred100 = xgb1.predict(test[feature_names])
sub = pd.read_csv('sample_submission.csv')
sub['target'] = pred100
sub['target'] = sub['target'].astype(int)
sub.to_csv('xgb_01_final_submission.csv', index=False)

In [ ]:
# Complete